In [1]:
# train a generative adversarial network on a one-dimensional function
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
import requests
import numpy as np
import pandas as pd
import json

from bs4 import BeautifulSoup


In [2]:
#file1 = open('cards-standard.txt', 'r') 
#Lines = file1.readlines() 
#maximum = len(Lines)
#cards_key = {}
#cards_value = {}
#i=1
#for line in Lines:
 #   cards_value[i] = line.rstrip("\n")
  #  cards_key[line.rstrip("\n")] = i
   # i += 1

#print(cards_key["Arcane Breath"])
#print(cards_value[32])
#print(len(Lines))

In [3]:
classes_id = {'Demon Hunter' : 16384,
          'Druid' : 4,
          'Hunter' : 8,
          'Mage' : 16,
          'Paladin' : 32,
          'Priest' : 64, 
          'Rogue' : 128,
          'Shaman' : 256,
          'Warlock' : 512,
          'Warrior' : 1024}

r = requests.get('https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&filter-class='+str(classes_id['Demon Hunter']))
soup = BeautifulSoup(r.content,"html.parser")
rows = soup.find('ul', class_="b-pagination-list paging-list j-tablesorter-pager j-listing-pagination")
links = rows.find_all('li')
print(links[len(links)-2].text)


57


In [10]:
#scrape deck data from Hearthpwn website, standard decks

def page_max(deck_type):
    r = requests.get('https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&filter-class='+str(classes_id[deck_type]))
    soup = BeautifulSoup(r.content,"html.parser")
    rows = soup.find('ul', class_="b-pagination-list paging-list j-tablesorter-pager j-listing-pagination")
    links = rows.find_all('li')
    return int(links[len(links)-2].text)

def scrape_decks(deck_type):
    start_urls = [] 
    n = page_max(deck_type)
    print('Number of Pages: ',n)
    for i in range(n):
        start_urls.append("https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=" + str(i+1)+"&filter-class="+ str(classes_id[deck_type]))
    #print(start_urls)
    deck_urls = []
    deck_classes = []
    for url in start_urls:
        print(url)
        r = requests.get(url)
        soup = BeautifulSoup(r.content,"html.parser")
        #rows = soup.find(id="decks-container")
 
        rows = soup.find_all('table', class_="listing listing-decks b-table b-table-a")
        for row in rows:
            decks = row.find_all('span', class_="tip")
            for deck in decks:
                links = deck.find_all('a', href=True)
                for link in links:
                    deck_urls.append("https://www.hearthpwn.com" + link['href'])
                    
            deck_class = row.find_all('td', class_="col-class")
            for cl in deck_class:
                deck_classes.append(cl.text)

    return cards_from_url(deck_urls), deck_classes



In [5]:
#scrape cards in deck from separate url
def cards_from_url(deck_urls):
    decks = []
    for url in deck_urls:
        print(url)
        deck=[]
        r = requests.get(url)
        soup = BeautifulSoup(r.content,"html.parser")
        table = soup.find_all('table', class_="listing listing-cards-tabular b-table b-table-a")
        for rows in table:
            row = rows.find_all('td', class_="col-name")
            for i in row:
                card = i.find('a')
                name = card.text
                number = card['data-count']

                for num in range(int(number)):
                    deck.append(name.replace('\n',''))
        #print(deck)
        #print(len(deck))
        decks.append(deck)
    return decks


        


In [6]:
def deck_to_vector(deck):
    deck_vector = []
    for elem in deck:
        #remove any potential white spaces from strings
        #if type(deck) is not list:
        elem = elem.lstrip()
        elem = elem.rstrip()

        deck_vector.append(cards_key[elem])
    return deck_vector

def vector_to_deck(deck_vector):
    deck = []
    for elem in deck_vector:
        deck.append(cards_value[elem])
    return deck

def vectorize_deck_list(decks):
    vecks = []
    for d in decks:
        vecks.append(deck_to_vector(d))
    return vecks

def concatenate_deck_class(decks, classes):
    concatenated_result = []
    for i in range(len(decks)):
        concatenated_result.append([decks[i],classes[i]])
    return concatenated_result



In [7]:
#deck_type = ['Demon Hunter','Druid','Hunter','Mage','Paladin','Priest','Rogue','Shaman','Warlock','Warrior']
#deck_type = ['Hunter','Mage','Paladin','Priest','Rogue','Shaman','Warlock']
#deck_type = ['Paladin','Priest','Rogue','Shaman','Warlock']

#for cl in deck_type:
 #   print("Scraping information: ", cl)
  #  decks, deck_classes = scrape_decks(cl)
   # print("Creating ", cl ," decks")
    #collected_decks = concatenate_deck_class(decks, deck_classes)
    #collected_decks_vectored = concatenate_deck_class(vectorize_deck_list(decks), deck_classes)
    #print("Creating ", cl, ".csv")
    #pd.DataFrame(collected_decks, columns=['cards', 'class']).to_csv("collected_decks_"+deck_type+".csv",index=False)
    #pd.DataFrame(collected_decks_vectored, columns=['cards', 'class']).to_csv("collected_decks_vectored_"+cl+".csv",index=False)



In [12]:
deck_type = ['Demon Hunter','Druid','Hunter','Mage','Paladin','Priest','Rogue','Shaman','Warlock','Warrior']
file = open("data.json", "r")
cards = json.load(file)


to_convert = []
for cl in deck_type:
    print("Scraping information: ", cl)
    decks, deck_classes = scrape_decks(cl)
    print("Creating ", cl ," decks")
    for deck in decks:
        for card in deck:
            card = next((new_card for new_card in cards if new_card['name'] == card), None)
    print("Creating ", cl, ".csv")
    to_convert.append(decks)
    #pd.DataFrame(decks, columns=['cards']).to_csv("collected_decks_"+cl+".csv",index=False)


#collected_decks_vectored = concatenate_deck_class(vectorize_deck_list(decks))
#pd.DataFrame(collected_decks, columns=['cards', 'class']).to_csv("collected_decks_"+deck_type+".csv",index=False)
#pd.DataFrame(collected_decks_vectored, columns=['cards', 'class']).to_csv("collected_decks_vectored_"+deck_type+".csv",index=False)



Scraping information:  Demon Hunter
Number of Pages:  57
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=1&filter-class=16384
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=2&filter-class=16384
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=3&filter-class=16384
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=4&filter-class=16384
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=5&filter-class=16384
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=6&filter-class=16384
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=7&filter-class=16384
https://www.hearthpwn.com/decks?f

https://www.hearthpwn.com/decks/1376324-illidari-aggro-dh
https://www.hearthpwn.com/decks/1376185-dmr-otk-demonhunter
https://www.hearthpwn.com/decks/1375860-tempo-dh
https://www.hearthpwn.com/decks/1376137-weapon-demon-hunter
https://www.hearthpwn.com/decks/1376105-fragmentos
https://www.hearthpwn.com/decks/1376086-patch-soul-dh
https://www.hearthpwn.com/decks/1376008-the-expendables
https://www.hearthpwn.com/decks/1375938-dh-otk
https://www.hearthpwn.com/decks/1375913-outcast-aggro-dh
https://www.hearthpwn.com/decks/1375779-super-demonetized-hunter
https://www.hearthpwn.com/decks/1375759-corrupt-dh-high-legend
https://www.hearthpwn.com/decks/1375757-legend-otk-demon-hunter-is-crazy
https://www.hearthpwn.com/decks/1375730-all-4-demonhunter
https://www.hearthpwn.com/decks/1375729-aggro-demon-hunter
https://www.hearthpwn.com/decks/1375405-3-old-god-demon-hunter
https://www.hearthpwn.com/decks/1375676-corrupted-highlander-dh-condi18
https://www.hearthpwn.com/decks/1375650-highlander-cthu

https://www.hearthpwn.com/decks/1371753-outcast-dh
https://www.hearthpwn.com/decks/1368980-legend-dh-midrange-token-df
https://www.hearthpwn.com/decks/1371658-darkmoon-soul-demon-hunter
https://www.hearthpwn.com/decks/1371651-1st-day-hero-demon-hunter
https://www.hearthpwn.com/decks/1371598-outcast-dh-is-more-than-a-meme
https://www.hearthpwn.com/decks/1371529-soul-fragment
https://www.hearthpwn.com/decks/1370516-token-dh
https://www.hearthpwn.com/decks/1370442-soul-demon-hunter
https://www.hearthpwn.com/decks/1371397-ilgynoth-otk
https://www.hearthpwn.com/decks/1371379-ilgynoth-demonhunter
https://www.hearthpwn.com/decks/1371373-strike-it-lucky
https://www.hearthpwn.com/decks/1371358-token-hunter
https://www.hearthpwn.com/decks/1370317-waffen-demonhunter
https://www.hearthpwn.com/decks/1371291-lifesteal-otk
https://www.hearthpwn.com/decks/1371264-budget-cthun-dh
https://www.hearthpwn.com/decks/1371259-outcast-aggro-fast-wins-budget
https://www.hearthpwn.com/decks/1371201-als
https://w

https://www.hearthpwn.com/decks/1364585-john-wicks-pencil
https://www.hearthpwn.com/decks/1364567-soulreader-tempo-to-legend-on-the-cheap
https://www.hearthpwn.com/decks/1364498-dh-you-are-not-prepared
https://www.hearthpwn.com/decks/1364322-tourney-deck
https://www.hearthpwn.com/decks/1364285-inner-demon-hunter
https://www.hearthpwn.com/decks/1364201-titans-highlander-demonhunter
https://www.hearthpwn.com/decks/1364193-standard-soul-dh-05-09-2020
https://www.hearthpwn.com/decks/1364135-highlander-dh
https://www.hearthpwn.com/decks/1364128-best-soul-dh-so-far
https://www.hearthpwn.com/decks/1363822-legend-illegal-dh-w-guide
https://www.hearthpwn.com/decks/1363921-dogshit1
https://www.hearthpwn.com/decks/1364057-big-dh
https://www.hearthpwn.com/decks/1364049-meta-sunfury
https://www.hearthpwn.com/decks/1364012-diamond-soul-demon-hunter-video-guide
https://www.hearthpwn.com/decks/1362709-76-wr-p6-to-leg-soul-tempo-dh
https://www.hearthpwn.com/decks/1363991-control-demon-hunter
https://ww

https://www.hearthpwn.com/decks/1359432-scholomance-soul-shard-tempo-dh
https://www.hearthpwn.com/decks/1359352-new-tempo
https://www.hearthpwn.com/decks/1359193-big
https://www.hearthpwn.com/decks/1359165-face-dh
https://www.hearthpwn.com/decks/1359146-aaa
https://www.hearthpwn.com/decks/1359138-soulcrusher
https://www.hearthpwn.com/decks/1359092-token-hunter
https://www.hearthpwn.com/decks/1359071-control-demons
https://www.hearthpwn.com/decks/1359065-sa-tempo-demon-hunter
https://www.hearthpwn.com/decks/1359023-xdd
https://www.hearthpwn.com/decks/1359022-xd
https://www.hearthpwn.com/decks/1358994-sa-tempo-dh-by-drtichk
https://www.hearthpwn.com/decks/1358965-shard-tempo-aggro
https://www.hearthpwn.com/decks/1358789-voracious-tokens-dh
https://www.hearthpwn.com/decks/1358775-soul-crusher-dh
https://www.hearthpwn.com/decks/1358773-sa-control-dh-by-drtichk
https://www.hearthpwn.com/decks/1358739-problem-test
https://www.hearthpwn.com/decks/1358686-demon-hunter
https://www.hearthpwn.com

...

https://www.hearthpwn.com/decks/1337701-pirate-warrior
https://www.hearthpwn.com/decks/1337620-outland-gala
https://www.hearthpwn.com/decks/1337605-highlander-warrior
https://www.hearthpwn.com/decks/1337575-enrage-warrior
https://www.hearthpwn.com/decks/1337528-aoo-theory-bomb-control
https://www.hearthpwn.com/decks/1337479-galakrond-warrior
https://www.hearthpwn.com/decks/1337448-aoo-bomb-warrior
https://www.hearthpwn.com/decks/1337441-control-warrior
https://www.hearthpwn.com/decks/1337401-gala-warrior
https://www.hearthpwn.com/decks/1337370-taunt-warrior
https://www.hearthpwn.com/decks/1337366-tempo-glakrond-highlander
https://www.hearthpwn.com/decks/1337298-aoo-theory-updated-galakrond
https://www.hearthpwn.com/decks/1337214-aoo-gready-big-warrior
https://www.hearthpwn.com/decks/1337194-taunt-bomb-warrior-ashes-of-outland
https://www.hearthpwn.com/decks/1336243-theorycraft-pirate-warrior
https://www.hearthpwn.com/decks/1337137-vdvsdvd
https://www.hearthpwn.com/decks/1337102-galakro

https://www.hearthpwn.com/decks/914247-fwewefwefw
https://www.hearthpwn.com/decks/911378-sindra
https://www.hearthpwn.com/decks/910035-123
https://www.hearthpwn.com/decks/909986-e
https://www.hearthpwn.com/decks/909689-import-warrior
https://www.hearthpwn.com/decks/873591-warrior-deck
https://www.hearthpwn.com/decks/871090-warrior
https://www.hearthpwn.com/decks/871086-agro-warrior
https://www.hearthpwn.com/decks/854592-zczcz
https://www.hearthpwn.com/decks/850918-f2p-aggro-pirate-deck
https://www.hearthpwn.com/decks/851071-6
https://www.hearthpwn.com/decks/849383-test
https://www.hearthpwn.com/decks/849839-basic-warrior-deck
https://www.hearthpwn.com/decks/846021-kvestt
https://www.hearthpwn.com/decks/842675-basic-aggro-pirate
https://www.hearthpwn.com/decks/840704-111111
https://www.hearthpwn.com/decks/834566-test
https://www.hearthpwn.com/decks/829409-aggro-basic-warrior
https://www.hearthpwn.com/decks/831519-aggo-wojownik
https://www.hearthpwn.com/decks/830509-standard-warrior
http

https://www.hearthpwn.com/decks/77927-maexxna-heroic-0-craft-cost
https://www.hearthpwn.com/decks/77476-f2p-nax-normal
https://www.hearthpwn.com/decks/79346-heroic-maexxna-charge-to-battle
https://www.hearthpwn.com/decks/76407-cheap-heroic-maexxna-warrior
https://www.hearthpwn.com/decks/75280-low-cost-fast-tempo-warrior
https://www.hearthpwn.com/decks/78126-heroic-maexxna-control-deck
https://www.hearthpwn.com/decks/77930-rage-warrior
https://www.hearthpwn.com/decks/77796-heroic-maexxna-battlecry
https://www.hearthpwn.com/decks/77629-vs-maexxna-v-0-9
https://www.hearthpwn.com/decks/77512-heroic-maexxna-w-no-rares-f2p
https://www.hearthpwn.com/decks/77464-maexxna-turtle-carnage
https://www.hearthpwn.com/decks/77261-one-deck-for-3-heroic-naxxramas-kills
https://www.hearthpwn.com/decks/77253-tronks-heroic-maexxna-deck
https://www.hearthpwn.com/decks/75372-my-take-on-aggro-warrior
https://www.hearthpwn.com/decks/75107-the-blood-haze
https://www.hearthpwn.com/decks/75106-the-blood-haze
http

https://www.hearthpwn.com/decks/36095-basic-warrior-aggro
https://www.hearthpwn.com/decks/35929-warrior-summoner
https://www.hearthpwn.com/decks/1376622-warrior
https://www.hearthpwn.com/decks/1375454-darcksize-guerriero
https://www.hearthpwn.com/decks/1374823-armor-bombs-and-weapons
https://www.hearthpwn.com/decks/1374056-turbo-etc
https://www.hearthpwn.com/decks/1373597-sawa
https://www.hearthpwn.com/decks/1372920-hard-rock-circus
https://www.hearthpwn.com/decks/1372873-ect-and-rattlegore
https://www.hearthpwn.com/decks/1371815-warrior
https://www.hearthpwn.com/decks/1370944-3-gods-warrior
https://www.hearthpwn.com/decks/1366986-big-warrior-cthun-the-shattered
https://www.hearthpwn.com/decks/1366836-new-combo-otk-armor-warrior-2020-ultra-top-tier-s
https://www.hearthpwn.com/decks/1366806-rattlegore
https://www.hearthpwn.com/decks/1366759-new-broken-warrior-deck-enrage-pirates-charge
https://www.hearthpwn.com/decks/1365340-kargath-kontrol
https://www.hearthpwn.com/decks/1364197-legend

https://www.hearthpwn.com/decks/356437-juggernautwarrior
https://www.hearthpwn.com/decks/268395-rush-warrior
https://www.hearthpwn.com/decks/268394-rush-warrior
https://www.hearthpwn.com/decks/268393-rush-warrior
https://www.hearthpwn.com/decks/268392-rush-warrior
https://www.hearthpwn.com/decks/261399-boom-troll
https://www.hearthpwn.com/decks/231456-0-dust-aggro-warrior
https://www.hearthpwn.com/decks/201695-wallet-warrior
https://www.hearthpwn.com/decks/134157-aggro-warrior
https://www.hearthpwn.com/decks/109330-tank-warrior-deck
https://www.hearthpwn.com/decks/105818-warrior-aggro-rush
https://www.hearthpwn.com/decks/103323-hulk-rage
https://www.hearthpwn.com/decks/93543-effective-control-warrior-made-by-herthike-ign
https://www.hearthpwn.com/decks/77171-rough-warrior
https://www.hearthpwn.com/decks/59706-jackmanics-fury-deck
https://www.hearthpwn.com/decks/58206-control-the-best-only-warrior-cards
https://www.hearthpwn.com/decks/55918-weapon-deck-no-pirates
https://www.hearthpwn.c

In [58]:

#strip of spaces
def strip(elem):
    elem = elem.rstrip()
    elem = elem.lstrip()
    return elem

#returns json of card name
def get_json(val):
    val = strip(val)
    for card in cards:
        card['name'] = strip(card['name'])
        if card['name'] == val:
            return card
    return '{}'
       

class_json = []
for class_ in to_convert:
    final_json=[]
    for deck in decks:
        deck_list=[]
        for card in deck:
            #print(card)
            json_card = get_json(card)
            #print(list(card.keys())[list(card.values()).index(card))
            #json_card = next((new_card for new_card in cards if new_card['name'] == card), None)
            #print(json_card['name'])
            deck_list.append(json_card)
        final_json.append(deck_list)
    class_json.append(final_json)

In [59]:
#explore the possibility of just doing json.dump like in Card-API
i=0
for i in range(len(class_json)):  
    with open('data_'+deck_type[i]+'.json', 'w', encoding='utf-8') as f:
        json.dump(class_json[i], f, ensure_ascii=False, indent=4)   